In [37]:
# Импорт библиотек
import numpy as n
import pandas as pd
import math
import CoolProp.CoolProp as CP
from bisect import bisect

# Греющий теплоноситель R236ea 

t11 = 50+273.15
t12 = 37+273.15
P1 = 0.243*10**6
G1 = 298
# Нагреваемый теплоноситель R236ea

t21 = 32+273.15
t22 = 41+273.15
P2 = 3.3*10**6
G2 = 298

koeff = 1.6 # (диапазон варьирования 1,25 - 2,5)

fluid = 'R236ea'
w2 = 1 # (диапазон варьирования 0,5 - 3 м/с)


# Теплофизические свойства греющей среды
H11 = CP.PropsSI('H','P',P1,'T',t11,fluid)
H12 = CP.PropsSI('H','P',P1,'T',t12,fluid)
T1 = (t11 +  t12)/2 
ro1 = CP.PropsSI('D','P',P1,'T',T1,fluid)
Cp1 = CP.PropsSI('C','P',P1,'T',T1,fluid)
lambd1 = CP.PropsSI('L','P',P1,'T',T1,fluid)
dinamic1 = CP.PropsSI('V','P',P1,'T',T1,fluid)
Pr1 = CP.PropsSI('PRANDTL','P',P1,'T',T1,fluid)
kinematic1 = dinamic1/ ro1
a1 = lambd1 / ( Cp1 * ro1)
H21 = CP.PropsSI('H','P',P2,'T',t21,fluid)
H22 = CP.PropsSI('H','P',P2,'T',t22,fluid)
T2 = (t21 +  t22)/2 
ro2 = CP.PropsSI('D','P',P2,'T',T2,fluid)
Cp2 = CP.PropsSI('C','P',P2,'T',T2,fluid)
lambd2 = CP.PropsSI('L','P',P2,'T',T2,fluid)
dinamic2 = CP.PropsSI('V','P',P2,'T',T2,fluid)
Pr2 = CP.PropsSI('PRANDTL','P',P2,'T',T2,fluid)
kinematic2 =  dinamic2 / ro2
a2 = lambd2 / ( Cp2 * ro2)
Q1 = G1 *(t11-t12)*Cp1/1000*0.99
Q2 = G2 *(t22-t21)*Cp2/1000
delta_tb = t11-t22
delta_tm = t12-t21
delta_t = (delta_tb - delta_tm) / n.log(delta_tb/delta_tm) #как обозначается натуральный логарифм???????
# поправка на схему движения
P = (t22-t21)/(t11-t21)
R = (t11-t12)/(t22-t21)
# зададимся скоростями движения теплоносителя
R_zagr = 0.0003 
# зададим геометрические характеристики ТО
# внутренний диаметр трубок
d_vn = 0.016 # м
# толщина стенки трубок
delta_st = 0.002
# наружный диаметр трубок
d_nar  = d_vn + 2 * delta_st
# диагональный межтрубный шаг

S = koeff * d_nar
S1 = (S**2 - (S/2)**2)**(1/2) # расстояние между рядами трубок 
f1 = (math.pi * d_vn**2)/4  # площадь поперечного сечения одной трубки
n_rash = G2 / (ro2 * w2 * f1) # примерное количество труб в одном ходе
n_fakt = math.ceil(n_rash)
w2_fakt = G2 / (ro2 * n_fakt * f1) #фактическая скорость фреона
lamb_steel = 26.1 # Вт/(м*с) - принимаем
# Определение коэффициента теплоотдачи со стороны нагреваемого теплоносителя
t_st2 = T2 #
Re2 = w2 * d_vn / kinematic2 #Безразмерный критерий подобия Рейнольдса для нагреваемого  теплоносителя
Nu2 = 0.021 * Re2**0.8 * Pr2**0.43 #Безразмерный критерий подобия Нуссельта для нагреваемого теплоносителя
alfa2 = Nu2 * lambd2 / d_vn
# Определение режима течения в межтрубном пространстве
D_vn = 1.1 * S * (n_fakt/0.6)**0.5*1000 #Внутренний диаметр кожуха, мм
if D_vn > 3000:
    D_vn = 3000
myList = [630, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000] 
myNumber = round(D_vn)
pos = (bisect(myList, myNumber))
myList[pos]         
D_vn = myList[pos]
D_vngost = D_vn/1000 # берем самое приблежнное значение из ряда по ГОСТ 9992-82
f_mtp = math.pi / 4 * (D_vngost**2 - d_nar**2 * n_fakt) # Площадь межтрубного пространства
fi = (1 - d_nar/S)/(1 - 0.9 *(d_nar/S)**2) # Коэффициент, учитывающий сужение живого сечения межтрубного пространства
lc = f_mtp/(D_vngost*(1-d_nar/S))
b = 0.2 * D_vngost #Расстояние от края сегментной перегородки до корпуса аппарата
L_ekv = lc + D_vngost - 4/3*b #Эквивалентная длина пути теплоносителя 
f_pr = f_mtp*lc*fi/ L_ekv
w1 = G1 / (ro1*f_mtp)
d_ekv = 4 * f_pr/( math.pi*(D_vngost + n_fakt*d_nar))
Re1 = w1 * d_ekv / kinematic1  #Безразмерный критерий Рейнольдса 
Nu1 = 0.023 * Re1**0.8 * Pr1**0.4
#принимаем значение температуры стенки 
t_st1 = T1
# Определение коэффициента теплоотдачи со стороны греющего теплоносителя
alfa1 = Nu1 * lambd1 / d_ekv
d_sr = (d_vn + d_nar) / 2
#коэффициент теплопередачи 
k = 1 / (d_sr*(1/(alfa1*d_nar) + 1/(2*lamb_steel)* n.log(d_nar/d_vn)+ 1/(alfa2*d_vn))+ R_zagr)
#Уточнение температуры стенки
q = k * delta_t
t_st1 = T1 - q / alfa1
t_st2 = T2 - q / alfa2
Pr_st1 = CP.PropsSI('PRANDTL','P',P1,'T',t_st1,fluid)
Pr_st2 = CP.PropsSI('PRANDTL','P',P2,'T',t_st2,fluid)
Nu2 = 0.021 * Re2**0.8 * Pr2**0.43 * (Pr2/Pr_st2)**0.25 #Безразмерный критерий подобия Нуссельта для нагреваемого теплоносителя
alfa2 = Nu2 * lambd2 / d_vn
#коэффициент теплопередачи 
k = 1 / (d_sr*(1/(alfa1*d_nar) + 1/(2*lamb_steel)* n.log(d_nar/d_vn)+ 1/(alfa2*d_vn)) + R_zagr)
#расчетная поверхность теплообмена 
F = Q1*1000 / (k* delta_t)
F_rash = F



# КОМПОНОВОЧНЫЙ РАЧЕТ 
sekciy = 2
print (sekciy, 'sekciy')
F_sekciy = F / sekciy
f1 = math.pi * d_vn**2 / 4 #Площадь проходного сечения одной трубки
n_xod = G2 / (ro2 * w2_fakt * f1) #Количество труб в одном ходе
# принимаем длину теплообменного аппарата 1, 1.5, 2, 3, 4, 6, 9 м
l = 9
print (l, 'l')
n_all = F_sekciy / (math.pi * d_sr * l) #Число труб в теплообменном аппарате 
n_all = math.ceil(n_all) 
L = F_sekciy / (math.pi * d_sr * n_xod) #общая длина трубок 
Z = L / l 
Z = math.ceil(Z) #число ходов  
print ('Z', Z, 'ходов')
n_xod1 = n_all / Z #Фактическое количество труб в одном ходе
n_xod1 =math.ceil(n_xod1) 
n_fakt_k = n_xod1 * Z #Фактическое количество труб в ТА в 1 секции
print ('n_fakt_k', n_fakt_k, 'труб в 1 секции')
w2_k = G2 / (ro2 * n_xod1 * f1) #Уточнение скорости нагреваемого теплоносителя 
Re2_k = w2_k * d_vn / kinematic2 #Безразмерный критерий подобия Рейнольдса для нагреваемого  теплоносителя
Nu2_k = 0.021 * Re2_k**0.8 * Pr2**0.43* (Pr2/Pr_st2)**0.25 #Безразмерный критерий подобия Нуссельта для нагреваемого теплоносителя
alfa2_k = Nu2_k * lambd2 / d_vn
l_fakt = F_sekciy / (math.pi * d_sr * n_fakt_k) #Уточнение длины труб в ТА
l_fakt = math.ceil(l_fakt)
#Трубы в трубных решетках размещены по сторонам шестиугольников.
a = ((12 * n_fakt_k - 3)**0.5 - 3) / 6 #Порядковый номер шестиугольника, считая от центра
a = math.ceil(a)
b = 2 * a-1 #Количество труб на диагонали шестиугольника
psi = 0.6 #коэффициент заполнения трубной решетки 
D_vn_k = 1.1 * S * (n_fakt_k/0.6)**0.5*1000 #Внутренний диаметр кожуха
if D_vn_k > 3000:
    D_vn_k = 3000
else:
    myList = [630, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000] 
    myNumber = round(D_vn_k)
    pos = (bisect(myList, myNumber))
    myList[pos]          
    D_vn_k = myList[pos]
D_vngost_k = D_vn_k/1000 # берем самое приблежнное значение из ряда по ГОСТ 9992-82
f_mtp_k = math.pi / 4 * (D_vngost_k**2 - d_nar**2 * n_fakt_k) # Площадь межтрубного пространства
lc_k =f_mtp_k/(D_vngost_k*(1-d_nar/S)) #Расстояние между сегментными перегородками
fi_k = (1 - d_nar/S)/(1 - 0.9 *(d_nar/S)**2) # Коэффициент, учитывающий сужение живого сечения межтрубного пространства

l_b = 0.2 * D_vngost_k # Расстояние от края сегментной перегородки до корпуса аппарата 
kol_per = math.ceil(l_fakt / lc_k)
if kol_per % 2 == 0:
    kol_per = kol_per
else:
    kol_per = kol_per + 1

L_ekv_k = lc_k + D_vngost_k - 4/3*l_b #Эквивалентная длина пути теплоносителя 
print ('D_vngost_k', D_vngost_k)

# f_pr = math.pi * D_vngost**2 / 4 - math.pi * d_nar**2 * n_fakt / 4  #Площадь сечения межтрубного пространства
f_pr_k = f_mtp_k*lc_k*fi_k/ L_ekv_k
w1_k = G1 / (ro1 * f_pr_k) # уточнение скорости греющего теплоносиетля
d_ekv_k = 4 * f_pr_k/( math.pi*(D_vngost_k + n_fakt_k*d_nar))
Re1_k = w1_k * d_ekv_k / kinematic1 #Безразмерный критерий Рейнольдса 
Nu1_k = 0.023 * Re1_k**0.8 * Pr1**0.4 # безразмерное число Нуссельта 
# Определение коэффициента теплоотдачи со стороны греющего теплоносителя
alfa1_k = Nu1_k * lambd1 / d_ekv_k
d2 = ((4 * G2) / (math.pi * ro2 * w2_k))**0.5 #Внутренний диаметр штуцера 
#коэффициент теплопередачи 
K_k = 1 / (d_sr*(1/(alfa1_k*d_nar) + 1/(2*lamb_steel)* n.log(d_nar/d_vn)+ 1/(alfa2_k*d_vn)) + R_zagr)
F_tepl = Q1*1000 / (K_k* delta_t)
l_tryb = math.ceil(F_tepl / (math.pi * n_fakt_k  * d_sr*sekciy))
print ('l_tryb', l_tryb)
F_fakt = math.pi * n_fakt_k * l_tryb * d_sr * sekciy #уточнение фактической площади
print ( 'F_fakt', F_fakt)
delta = (F_fakt - F_tepl) / F_fakt * 100


  # ГИДРАВЛИЧЕСКИЙ РАСЧЕТ
# расчет потери давления в трубном пространстве 
w_tr = G2 * Z / (0.785 * d_vn**2 *n_fakt_k*ro2) #скорость теплоносителя в трубах
deltaa = 0.0002 #высота выступов шероховатости (для стальных труб можно принять ∆ = 0,2 мм)
e = deltaa / d_vn #относительная шероховатость труб
lambd =0.25 * (n.log10(e/3.7 + (6.81/Re2_k)**0.9)**(-2)) #Коэффициент трения при Reтр > 2300 можно определить по выражению
#Коэффициенты местных сопротивлений потоку, движущемуся в трубном пространстве
ksi_1 = 1.5 #входная и выходная камера
ksi_2 = 2.5 #поворот между ходами
ksi_3 = 1 #вход в трубы и выход из них
w_tr_shtytcera = G2 / (0.785 * d2**2 *ro2)
#Местные сопротивления на входе в распределительную камеру и на выходе из нее следует рассчитывать по скорости потока в штуцерах wтр шт
delta_P_tr = lambd * l_fakt* Z / d_vn *  w_tr**2 /2 * ro2 + (2.5*(Z-1+2*Z*w_tr**2 /2 * ro2) + 3* w_tr_shtytcera**2 /2 * ro2)
delta_P_tr = round (delta_P_tr, 3)
#Расчет потери давления в межтрубном пространстве 
w_mtr = G1 / (ro1 * f_pr_k) #Скорость жидкости в межтрубном пространстве
# Коэффициенты местных сопротивлений потоку, движущемуся в межтрубном пространстве
sigma_mtr1 = 1.5 # вход и выход жидкости 
sigma_mtr2 = 1.5 # поворот через сегментную перегородку
m = (n_fakt_k/3)**0.5 # число рядов труб, которое приближенно можно определить по выражению
sigma_mtr3 = 3*m/Re1_k**0.2 #сопротивление пучка труб
d_mtr_shtytcera = ((4 * G1) / (math.pi * ro1 * w1_k))**0.5 #внутренний диаметр штуцера 
w_mtr_shtytcera = G1  / (0.785 * d_mtr_shtytcera**2 * ro1)
delta_P_mtr = 3 *m*(kol_per+1)*w_mtr**2 /2 / Re1_k**0.2 * ro1 + 1.5 * kol_per*w_mtr**2 /2 *ro1 + 3 * w_mtr_shtytcera**2 / 2 *ro1
# delta_P_mtr = round(delta_P_mtr, 3)
print ('delta_P_tr',  (delta_P_tr/1000), 'kПа')
print ('delta_P_mtr', (delta_P_mtr/1000), 'kПа')
print (alfa1_k, alfa2_k)


#Масса ТО

 #Толщина цилиндрических обечаек 
s = 4

# Расчет массы аппарата 
print ('Масса цилиндрической части')
D_vnesh = D_vngost_k + 2 * s / 1000 
print ('D_vnesh', D_vnesh)
# Плотность стали 
ro_stal = 7832 # кг/м3
# объем цилиндрической части 
V_cil = math.pi * (D_vnesh**2-D_vngost_k**2)/4 * l_tryb /1000
print ('V_cil', V_cil)
massa_cil = V_cil * ro_stal
print ('Масса цилиндрической части', massa_cil, 'кг')
m_tryb = (d_nar**2 - d_vn**2)/4 * math.pi * l_tryb * ro_stal * n_fakt_k
print ('Масса труб',m_tryb, 'кг')
Massa =   massa_cil + m_tryb 
print ('Масса', Massa, 'кг')

2 sekciy
9 l
Z 2 ходов
n_fakt_k 1936 труб в 1 секции
D_vngost_k 2.0
l_tryb 7
F_fakt 1532.6950910921598
delta_P_tr 50.366087 kПа
delta_P_mtr 96.33509071819324 kПа
517.0865694191764 1617.1553701705036
Масса цилиндрической части
D_vnesh 2.008
V_cil 0.00017628104697823099
Масса цилиндрической части 1.380633159933505 кг
Масса труб 12004.067953433796 кг
Масса 12005.44858659373 кг
